In [37]:
import re
import pandas as pd
import numpy as np
import spacy

In [38]:
def clean(text): 
    text = re.sub('[0-9]+.\t' + '...','',str(text))
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    text = re.sub("'s",'',str(text))
    text = re.sub("-",' ',str(text))
    text = re.sub("—",'',str(text))
    text = re.sub('\"','',str(text))
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return text

In [39]:
def sentences(text):
    text = re.split('adwd adaldkjaldk jenjfbakdnjak jhbaiauwhd aonlda', text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent

In [40]:
# Dataframe
df = pd.read_csv(r'.\datasets_AI\news.csv')
df = df.drop(columns=['url', 'hostname', 'timestamp'], axis=1)
df = df[df.story == 'dABGVITQs6X1I4MdYGnX9zY59PpVM']
# clean speech
df['Speech_clean'] = df['main_content'].apply(clean)

# seperate sentences
df['sent'] = df['Speech_clean'].apply(sentences)
# temporary disable


#reset indices for removed stuff
df.reset_index(inplace=True)
df.drop(['Unnamed: 0', 'index'], axis = 1, inplace = True)

df.head()

,id,title,publisher,category,story,main_content,main_content_len,Speech_clean,sent
0,22237,"The Incredibles 2, Cars 3 in the works, Disney...",Digital Spy,e,dABGVITQs6X1I4MdYGnX9zY59PpVM,The Incredibles 2 and Cars 3 are in developmen...,1340.0,The Incredibles 2 and Cars 3 are in developmen...,[The Incredibles 2 and Cars 3 are in developme...
1,22241,The Incredibles are set for another big-screen...,Belfast Telegraph,e,dABGVITQs6X1I4MdYGnX9zY59PpVM,"Incredibles 2, Cars 3 in the works BelfastTele...",1620.0,"Incredibles 2, Cars 3 in the works BelfastTele...","[Incredibles 2, Cars 3 in the works BelfastTel..."
2,22244,Pixar Working On Sequels For Popular Animated ...,Online News Heard Now,e,dABGVITQs6X1I4MdYGnX9zY59PpVM,Posted by News\n\nPixar Working On Sequels For...,1339.0,Posted by News Pixar Working On Sequels For P...,[Posted by News Pixar Working On Sequels For ...
3,22248,"State Of The (Disney) Union: Cars 3, Incredibl...",Contactmusic.com,e,dABGVITQs6X1I4MdYGnX9zY59PpVM,In news you didn’t know you needed until right...,1737.0,In news you didn’t know you needed until right...,[In news you didn’t know you needed until righ...
4,22249,Disney Pixar confirm The Incredibles 2,Total Film,e,dABGVITQs6X1I4MdYGnX9zY59PpVM,The first footage from Incredibles 2 (there's ...,2942.0,The first footage from Incredibles 2 was show...,[The first footage from Incredibles 2 was sho...


In [41]:
df2 = pd.DataFrame(columns=['sent','id','len'])

row_list = []

for i in range(len(df)):
    for sent in df.at[i,'sent']:
    
        wordcount = len(sent.split())
        id = df.at[i,'id']

        dict1 = {'id' : id, 'sent' : sent, 'len' : wordcount}
        row_list.append(dict1)
    

df2 = pd.DataFrame(row_list)
df2.head()

,id,sent,len
0,22237,The Incredibles 2 and Cars 3 are in developmen...,224
1,22241,"Incredibles 2, Cars 3 in the works BelfastTele...",239
2,22244,Posted by News Pixar Working On Sequels For P...,227
3,22248,In news you didn’t know you needed until right...,306
4,22249,The first footage from Incredibles 2 was show...,503


In [42]:
nlp = spacy.load('en_core_web_md')

In [43]:
p = df2['sent'].tolist()
docx = nlp(p[np.random.randint(1, len(df2))])
docy = nlp(p[np.random.randint(1, len(df2))])
x = docx.similarity(docy)

[(docx, docy), x]

[(Disney owned animation studio Pixar is to produce a sequel to the Oscar winning animation The Incredibles, more than a decade after Brad Bird hugely popular tale of a family of retired superheroes first hit cinemas.  The new instalment was announced yesterday by Walt Disney Bob Iger at a briefing for shareholders in Portland, Oregon. Iger also said Pixar would work on a second sequel to 2006 Cars.  The Incredibles is set in a world in which superheroes exist but have been banned from crime fighting. Mr Incredible  has been living incognito as plain old Bob Parr when a mysterious offer gives him the chance to return to action. Soon he and his wife Helen , along with their three children , are caught up in a not entirely welcome new adventure.  Bird followed up The Incredibles, his first Pixar film, with 2007 Ratatouille, winning the Oscar for best animation for both films. They are considered highlights from a golden era of animation, between Finding Nemo in 2003 and Toy Story 3 in 20

In [44]:
def dictfy(d1, t1):
    sendict = dict()
    for key in t1:
        sendict[key] = []
        for word in d1:
            if word.label_ == key:
                sendict[key].append(str(word).lower().strip())
    for key in sendict.keys():
        sendict[key] = list(set(sendict[key]))
    return sendict


In [45]:
if x > 0.925:
    x1 = dictfy([ent for ent in docx.ents], [ent.label_ for ent in docx.ents])
    xres = filter(lambda x: x.tag_ == 'VBG', docx)
    x1['VERB'] = list(set(xres))
    y1 = dictfy([ent for ent in docy.ents], [ent.label_ for ent in docy.ents])
    yres = filter(lambda y: y.tag_ == 'VBG', docy)
    y1['VERB'] = list(set(yres))
    print([x1, y1], x)
else:
    print([(docx, docy), x], "sentences not similar enough")

# [[ent.text, ent.label_] for ent in docy.ents]

[{'ORG': ['disney', 'incredibles', 'iger', 'incredibles', 'incredibles', 'walt disney', 'the la times', 'disney california adventure', 'disney', 'monsters university'], 'PERSON': ['pixar', 'oscar', 'brad bird', 'walt disney bob iger', 'pixar', 'bob parr', 'helen', 'pixar', 'oscar', 'nemo', 'pixar', 'pixar', 'oscar', 'rotten tomatoes', 'nemo'], 'DATE': ['more than a decade', 'yesterday', '2006', '2003', '2010', 'a poor year', '2010', 'last year', 'this summer', '2013', '2016'], 'ORDINAL': ['first', 'second', 'first', 'first'], 'GPE': ['portland', 'oregon', 'anaheim'], 'CARDINAL': ['three', '3', 'six', '2'], 'PERCENT': ['39%'], 'MONEY': ['billions of dollars'], 'VERB': [living, according, producing, Finding, winning, Finding, winning]}, {'ORG': ['cars and the incredibles', 'variety', 'disney', 'incredibles', 'incredibles', 'toy story', 'incredibles', 'toy story', 'monsters inc.', 'cars and finding nemo', 'finding dory'], 'PERSON': ['bob iger', 'brad bird', 'pixar', 'pixar', 'garth ennis'